# Stock Price Modeling from Fundamentals

In [1198]:
# Todo :
# - Create labels from stock price data before and after earnings. 
#   This should correlate the change in stock price with the change in fundamentals
# - Create validation and test sets
#   4th most recent quarter's price for validation.
#   3 most recent quarters' price for test
# - Start modeling. Try some baselines first (e.g. mean estimates, repeating previous periods)
#   Try some additional models (e.g. ARIMA, XGBoost trees)

In [1199]:
from pathlib import Path
import pandas as pd

prices = pd.read_csv(Path("preprocessed") / "KMB_prices_dates.csv")
financials = pd.read_csv(Path("preprocessed") / "KMB_financials.csv")

prices.shape, financials.shape

((6282, 6), (448, 319))

In [1200]:
prices["target"] = prices["Close"] - prices["Open"]
prices = prices[["Date", "target"]] # backward looking
priced = prices.sort_values(by="Date")
prices.head(3)

,Date,target
0,2001.144809,0.083130
1,2001.147541,-0.062352
2,2001.155738,0.257708


In [1201]:
financials["period_end"] = financials["period_end"] 
financials.head(3)

,period_end,filing_date,form,AccountsPayableCurrent,AccountsPayableTradeCurrent,AccountsReceivableNetCurrent,AccruedAdvertisingCurrent,AccruedIncomeTaxesNoncurrent,AccruedLiabilitiesCurrent,AccruedSalariesCurrent,...,UnrecordedUnconditionalPurchaseObligationBalanceOnThirdAnniversary,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesIssuedBasic,WeightedAverageNumberOfSharesOutstandingBasic,0,1,2,3
0,2006.997268,2010.150273,10-K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
1,2007.997268,2009.598361,10-Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
2,2007.997268,2009.846995,10-Q/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0


In [1202]:
merged = prices.merge(financials, left_on="Date", right_on="period_end").sort_values(by="period_end")
merged.head(3)

,Date,target,period_end,filing_date,form,AccountsPayableCurrent,AccountsPayableTradeCurrent,AccountsReceivableNetCurrent,AccruedAdvertisingCurrent,AccruedIncomeTaxesNoncurrent,...,UnrecordedUnconditionalPurchaseObligationBalanceOnThirdAnniversary,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesIssuedBasic,WeightedAverageNumberOfSharesOutstandingBasic,0,1,2,3
0,2007.997268,-0.162019,2007.997268,2009.598361,10-Q,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
1,2007.997268,-0.162019,2007.997268,2009.846995,10-Q/A,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0
2,2007.997268,-0.162019,2007.997268,2010.150273,10-K,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0


In [1203]:
dates = pd.read_csv(Path("preprocessed") / "KMB_dates.csv")
dates.head(3)

,filing_date,report_date
0,2026.117486,2025.997268
1,2026.109290,2026.109290
2,2026.101093,2025.245902


In [1204]:
merged = merged.merge(dates, left_on="Date", right_on="report_date", suffixes=("", "_drop"))
merged.shape

(202, 323)

In [1205]:
X = merged.drop(columns=["target", "Date", "filing_date", "report_date"]) # filing date can reflect some arbitrary date in the future.
X = X.rename(columns={"filing_date_drop": "filing_date"})
y = merged["target"]

X.shape, y.shape

((202, 319), (202,))

In [1206]:
X.head()

,period_end,form,AccountsPayableCurrent,AccountsPayableTradeCurrent,AccountsReceivableNetCurrent,AccruedAdvertisingCurrent,AccruedIncomeTaxesNoncurrent,AccruedLiabilitiesCurrent,AccruedSalariesCurrent,AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,...,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesIssuedBasic,WeightedAverageNumberOfSharesOutstandingBasic,0,1,2,3,filing_date
0,2016.748634,10-Q,2.454000e+09,NaN,2.222000e+09,NaN,NaN,1.731000e+09,NaN,9.729000e+09,...,NaN,NaN,361950000.0,NaN,359600000.0,0,1,0,0,2016.814208
1,2016.748634,10-Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,361950000.0,NaN,359600000.0,0,1,0,0,2016.814208
2,2017.087432,10-K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,0,0,0,2017.090164
3,2017.245902,10-Q,2.571000e+09,NaN,2.224000e+09,NaN,NaN,1.620000e+09,NaN,9.940000e+09,...,NaN,NaN,358600000.0,NaN,356000000.0,0,1,0,0,2017.311475
4,2017.245902,10-Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,358600000.0,NaN,356000000.0,0,1,0,0,2017.311475


In [1207]:
mask = X["period_end"] >= 2025
X_train, X_test = X.loc[~mask, :], X.loc[mask, :]
y_train, y_test = y.loc[~mask], y.loc[mask]

X_train.shape, y_train.shape, X_test.shape, y_test.shape 

((158, 319), (158,), (44, 319), (44,))

In [1208]:
# test 4 different events for validation, 6 events for test 
X_val, X_test = X_test.loc[:196,:], X_test.loc[197:,:]
y_val, y_test = y_test.loc[:196], y_test.loc[197:]

X_val.shape, y_val.shape, X_test.shape, y_test.shape,

((39, 319), (39,), (5, 319), (5,))

# Prediction Task  
  
Predict the day-of-earnings price movement (i.e. close price minus open price)

## Baseline

In [1209]:
mean_pred = y_train.mean()
max_pred = y_train.max()
med_pred = y_train.median()
recent_pred = y_train.iloc[-1]

mean_pred, max_pred, med_pred, recent_pred,

(np.float64(0.4280892238119912),
 np.float64(1.4560128717191958),
 np.float64(0.4447440844507611),
 np.float64(-0.9807497356786712))

# CONTINUE HERE - baseline modeling, then some more advanced models.

In [1210]:
import numpy as np
from sklearn.metrics import root_mean_squared_error as rmse

def print_rmse(actual, pred, loss_for):
    loss = rmse(actual, pred)

    print(f"{loss_for} loss = {loss:.4f}")

def print_full_rmse(const_pred):
    print_rmse(y_train, y_train.apply(lambda x: const_pred), "Training")
    print_rmse(y_val, y_val.apply(lambda x: const_pred), "Validation")
    print_rmse(y_test, y_test.apply(lambda x: const_pred), "Test")

In [1211]:
print("Mean baseline:")
print_full_rmse(mean_pred)
print()
print("Max baseline:")
print_full_rmse(max_pred)
print()
print("Median baseline:")
print_full_rmse(med_pred)
print()
print("Recent baseline:")
print_full_rmse(recent_pred)
print()

Mean baseline:
Training loss = 0.9128
Validation loss = 0.5238
Test loss = 1.0805

Max baseline:
Training loss = 1.3747
Validation loss = 1.3536
Test loss = 1.2622

Median baseline:
Training loss = 0.9130
Validation loss = 0.5317
Test loss = 1.0759

Recent baseline:
Training loss = 1.6787
Validation loss = 1.2538
Test loss = 2.0043



To contextualize this RMSE, the current stock price is ~$100 so the loss approximately corresponds to the percent error in estimated and actual pricing.  

## SARIMA

In [1212]:
X.head(3)

,period_end,form,AccountsPayableCurrent,AccountsPayableTradeCurrent,AccountsReceivableNetCurrent,AccruedAdvertisingCurrent,AccruedIncomeTaxesNoncurrent,AccruedLiabilitiesCurrent,AccruedSalariesCurrent,AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,...,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesIssuedBasic,WeightedAverageNumberOfSharesOutstandingBasic,0,1,2,3,filing_date
0,2016.748634,10-Q,2.454000e+09,NaN,2.222000e+09,NaN,NaN,1.731000e+09,NaN,9.729000e+09,...,NaN,NaN,361950000.0,NaN,359600000.0,0,1,0,0,2016.814208
1,2016.748634,10-Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,361950000.0,NaN,359600000.0,0,1,0,0,2016.814208
2,2017.087432,10-K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,0,0,0,2017.090164


In [1213]:
merged = merged.drop(columns=["Date", "filing_date_drop", "report_date"])
merged.shape

(202, 320)

In [1214]:
merged.head(3)

,target,period_end,filing_date,form,AccountsPayableCurrent,AccountsPayableTradeCurrent,AccountsReceivableNetCurrent,AccruedAdvertisingCurrent,AccruedIncomeTaxesNoncurrent,AccruedLiabilitiesCurrent,...,UnrecordedUnconditionalPurchaseObligationBalanceOnThirdAnniversary,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesIssuedBasic,WeightedAverageNumberOfSharesOutstandingBasic,0,1,2,3
0,0.312489,2016.748634,2016.814208,10-Q,2.454000e+09,NaN,2.222000e+09,NaN,NaN,1.731000e+09,...,NaN,NaN,NaN,361950000.0,NaN,359600000.0,0,1,0,0
1,0.312489,2016.748634,2017.808743,10-Q,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,361950000.0,NaN,359600000.0,0,1,0,0
2,0.358983,2017.087432,2017.106557,10-K,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0


In [1215]:
financial_cols = list(set(merged.columns) - {'0', '1', '2', '3', 'filing_date', 'form'})
binarized_form_cols = ['0', '1', '2', '3']

len(financial_cols)

314

In [1216]:
grouped = merged.groupby('period_end')

agg_logic = {col: 'first' for col in financial_cols}
form_logic = {col: 'max' for col in binarized_form_cols}

df_consolidated = grouped.agg({**agg_logic, **form_logic})
df_consolidated.head(3)

,UnrecordedUnconditionalPurchaseObligationBalanceOnThirdAnniversary,UnrecordedUnconditionalPurchaseObligationDueAfterFiveYears,BusinessCombinationStepAcquisitionEquityInterestInAcquireeRemeasurementGain,EquityMethodInvestments,SalesRevenueNet,GainFromHedgedFirmCommitmentNotQualifyingAsFairValueHedge,UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities,ReceivablesFromCustomers,OtherReceivables,DeferredIncomeTaxLiabilitiesNet,...,EntityPublicFloat,DeferredIncomeTaxExpenseBenefit,NotionalAmountOfInterestRateFairValueHedgeDerivatives,OtherComprehensiveIncomeLossNetOfTax,EarningsPerShareBasic,OperatingLeasesFutureMinimumPaymentsDueInThreeYears,0,1,2,3
period_end,,,,,,,,,,,,,,,,,,,,,
2016.748634,NaN,NaN,NaN,289000000.0,9.126000e+09,0.0,NaN,NaN,NaN,NaN,...,NaN,-13000000.0,NaN,124000000.0,3.07,NaN,0,1,0,0
2017.087432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
2017.245902,NaN,NaN,NaN,284000000.0,4.483000e+09,0.0,NaN,NaN,NaN,NaN,...,NaN,-25000000.0,NaN,249000000.0,1.58,NaN,0,1,0,0


In [1217]:
merged.notna().sum().sum(), merged.notna().sum().sum() / merged.size

(np.int64(14097), np.float64(0.21808477722772276))

In [1218]:
df_consolidated.notna().sum().sum(), df_consolidated.notna().sum().sum() / df_consolidated.size

(np.int64(3548), np.float64(0.35991073240008115))

In [1219]:
X_consolidated = df_consolidated.drop(columns=["target"])
y_consolidated = df_consolidated["target"]

X_consolidated.shape, y_consolidated.shape

((31, 317), (31,))

In [1220]:
from sklearn.decomposition import TruncatedSVD

X_zero = X_consolidated.fillna(0) # ignored in sparse representation

pca = TruncatedSVD(n_components=25) # avoid high dimensional data by using <= 31 features
X_low_dim = pca.fit_transform(X_zero)
X_low_dim = pd.DataFrame(X_low_dim)
X_low_dim.shape

(31, 25)

In [1221]:
dates = X_consolidated["period_end"]
years = dates.astype(int)
days =(dates - years)

days.head(3)

period_end
2016.748634    0.748634
2017.087432    0.087432
2017.245902    0.245902
Name: period_end, dtype: float64

In [1222]:
def coord_to_date(year, fractional_day):
    day_of_year = int(fractional_day * 366)
    return pd.to_datetime(year, format='%Y') + pd.Timedelta(days=day_of_year - 1)

# Apply this to create a real DatetimeIndex
X_low_dim.index = [coord_to_date(y, f) for y, f in zip(years, days)]
X_low_dim.head(3)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
2016-09-29,9.903434e+05,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-9.236589e+05,9.574977e+04
2017-01-31,1.606226e-09,-1.264296e-08,9.377327e-05,-2.411411e-05,-3.951088e-05,2.225139e-05,-2.164786e-05,2.900490e-05,-8.847186e-06,-2.652633e-05,...,-1.919998e-04,-5.064913e-06,-9.107672e-05,-4.534803e-05,1.153421e-04,2.034103e-04,3.562749e+08,1.305410e-04,9.028331e-05,1.319096e-04
2017-03-31,9.833006e+05,1.561090e+07,3.015532e+10,-7.961848e+09,-1.284246e+10,8.659599e+09,-5.809053e+09,8.111368e+08,5.850621e+08,8.528219e+08,...,-4.556031e+08,-1.636809e+08,1.199170e+09,3.540245e+08,-1.165515e+08,-2.504732e+08,7.451071e-04,-8.897524e+06,6.376540e+06,1.405834e+06


In [1223]:
full_calendar = pd.date_range(start=X_low_dim.index.min(), end=X_low_dim.index.max(), freq='D')
daily = X_low_dim.reindex(full_calendar)
daily = daily.ffill()

daily

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
2016-09-29,9.903434e+05,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-9.236589e+05,9.574977e+04
2016-09-30,9.903434e+05,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-9.236589e+05,9.574977e+04
2016-10-01,9.903434e+05,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-9.236589e+05,9.574977e+04
2016-10-02,9.903434e+05,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-9.236589e+05,9.574977e+04
2016-10-03,9.903434e+05,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-9.236589e+05,9.574977e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,1.259133e+09,-2.003479e+07,3.714720e+10,-1.166539e+10,-4.664851e+09,-3.623203e+09,3.371281e+09,2.319259e+09,-9.340430e+07,-9.981908e+08,...,-8.682214e+08,1.884232e+07,-1.111847e+07,-3.186704e+08,1.885047e+08,-1.069887e+08,-3.641362e-04,-1.794402e+08,-1.410438e+08,1.887345e+08
2025-12-28,1.259133e+09,-2.003479e+07,3.714720e+10,-1.166539e+10,-4.664851e+09,-3.623203e+09,3.371281e+09,2.319259e+09,-9.340430e+07,-9.981908e+08,...,-8.682214e+08,1.884232e+07,-1.111847e+07,-3.186704e+08,1.885047e+08,-1.069887e+08,-3.641362e-04,-1.794402e+08,-1.410438e+08,1.887345e+08
2025-12-29,1.259133e+09,-2.003479e+07,3.714720e+10,-1.166539e+10,-4.664851e+09,-3.623203e+09,3.371281e+09,2.319259e+09,-9.340430e+07,-9.981908e+08,...,-8.682214e+08,1.884232e+07,-1.111847e+07,-3.186704e+08,1.885047e+08,-1.069887e+08,-3.641362e-04,-1.794402e+08,-1.410438e+08,1.887345e+08
2025-12-30,1.259133e+09,-2.003479e+07,3.714720e+10,-1.166539e+10,-4.664851e+09,-3.623203e+09,3.371281e+09,2.319259e+09,-9.340430e+07,-9.981908e+08,...,-8.682214e+08,1.884232e+07,-1.111847e+07,-3.186704e+08,1.885047e+08,-1.069887e+08,-3.641362e-04,-1.794402e+08,-1.410438e+08,1.887345e+08


In [1224]:
!pip install statsmodels

In [1225]:
y = pd.read_csv(Path("preprocessed") / "KMB_prices_floats.csv").set_index("Date")
y = y.set_index(pd.to_datetime(y.index))
y_stock_returns = (y["Close"] - y["Open"])
y_stock_returns = y_stock_returns.loc[daily.index.min():daily.index.max()]
daily = daily.loc[y_stock_returns.index]
print(y_stock_returns.shape, daily.shape)
y_stock_returns.head(3)

(2327,) (2327, 25)


Date
2016-09-29   -0.646778
2016-09-30    0.312489
2016-10-03   -0.552301
dtype: float64

In [1226]:
daily.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-29,990343.364731,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-923658.871474,95749.772607
2016-09-30,990343.364731,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-923658.871474,95749.772607
2016-10-03,990343.364731,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-923658.871474,95749.772607
2016-10-04,990343.364731,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-923658.871474,95749.772607
2016-10-05,990343.364731,1.623384e+07,3.064898e+10,-8.328908e+09,-1.167468e+10,1.287505e+10,-5.405903e+09,2.965022e+09,1.476419e+08,-2.623201e+09,...,2.766858e+08,1.143629e+08,-4.416639e+08,-1.392304e+08,5.069331e+07,1.134582e+08,9.531910e-08,7.165647e+06,-923658.871474,95749.772607


In [1227]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(y_stock_returns, 
                exog=daily, 
                order=(1, 0, 1))

results = model.fit(low_memory=True, disp=False)

c:\Users\seani\miniforge3\envs\kmb_analysis\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\seani\miniforge3\envs\kmb_analysis\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\seani\miniforge3\envs\kmb_analysis\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\seani\miniforge3\envs\kmb_analysis\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting

In [ ]:
results